# Contents
2.1 [The SELECT Statement and Clauses](#2.1-The-SELECT-Statement-and-Clauses) <br>
&emsp;2.1.1 [Order of the SELECT Statement Clauses](#2.1.1-Order-of-the-SELECT-Statement-Clauses) <br>
&emsp;2.1.2 [Clause Execution Order](#2.1.2-Clause-Execution-Order) <br>
2.2 [Overview of Data Types](#2.2-Overview-of-Data-Types) <br>
&emsp;2.2.1 [Numeric Data](#2.2.1-Numeric-Data) <br>
&emsp;2.2.2 [Character Data](#2.2.2-Character-Data) <br>
2.3 [SQL Keywords](#2.3-SQL-Keywords) <br>
&emsp;2.3.1 [Creating Column Aliases](#2.3.1-Creating-Column-Aliases) <br>
&emsp;2.3.2 [Finding Duplicate (Unique) Values](#2.3.2-Finding-Duplicate-(Unique)-Values) <br>
2.4 [SQL Operators, Functions, and Keywords](#2.4-SQL-Operators,-Functions,-and-Keywords) <br>
&emsp;2.4.1 [Comparison Operators](#2.4.1-Comparison-Operators) <br>
&emsp;2.4.2 [Logical Operators](#2.4.2-Logical-Operators) <br>
&emsp;2.4.3 [Arithmetic Operators](#2.4.3-Arithmetic-Operators) <br>
&emsp;2.4.4 [Character String Operators and Functions](#2.4.4-Character-String-Operators-and-Functions) <br>
&emsp;2.4.5 [Summarizing Data](#2.4.5-Summarizing-Data) <br>
&emsp;2.4.6 [Predicates](#2.4.6-Predicates) 

In [1]:
# Left-align all the tables in any markdown
from IPython.core.display import HTML
table_css = 'table {align:left;display:block} '
HTML('<style>{}</style>'.format(table_css))

In [2]:
import saspy               # Import the saspy module
sas = saspy.SASsession()   # Connect to the SAS environment

Using SAS Config named: oda
SAS Connection established. Subprocess id is 29812



In [3]:
%%SAS
libname sql '~/sql';   *Assign a library reference;
title;                 *No title for all procedures output;

Using SAS Config named: oda
SAS Connection established. Subprocess id is 18544



### 2.1 The SELECT Statement and Clauses
The `SELECT` statement is often referred to as an **SQL query** and is used to read data from one ore more database tables (or data sets). It has only one clause that is required to be specified -- the `FROM` clause. 

#### 2.1.1 Order of the SELECT Statement Clauses
1. SQL = `SELECT`
2. Is  = `INTO`
3. Fun = `FROM`
4. When   = `WHERE`
5. Geeks  = `GROUP BY`
6. Help  = `HAVING`
7. Others = `ORDER BY`

In [4]:
%%SAS
proc sql;
    select prodname,
           prodtype,
           prodcost
        into :m_prodname,
             :m_prodtype,
             :m_prodcost
            from sql.products
                where prodname contains 'Software'
                    group by prodtype
                        having count(prodtype) > 3
                            order by prodname;
quit;

Product Name,Product Type,Product Cost
Database Software,Software,$399.00
Graphics Software,Software,$299.00
Spreadsheet Software,Software,$299.00
Wordprocessor Software,Software,$299.00


#### 2.1.2 Clause Execution Order
<table>
  <tr>
    <th>Clause Execution Order </th>
    <th>Description</th>
  </tr>
  <tr>
    <td>1. FROM Clause</td>
    <td>The FROM clause is a required clause with the purpose of determining the working set of data that is being queried.</td>
  </tr>
  <tr>
    <td>2. INTO Clause</td>
    <td>The INTO clause is used to create one or more macro variables where the values can be used to manipulate data in DATA and PROC steps.</td>
  </tr>
    <tr>
    <td>3. WHERE Clause</td>
    <td>The WHERE clause is used to subset rows of data based on the condition(s) specified, and rows that aren't satisfied by the condition(s) are discarded.</td>
  </tr>
    <tr>
    <td>4. GROUP BY Clause</td>
    <td>The GROUP BY clause takes the rows that were subset with the WHERE clause and grouped based on common values in the column specified in the GROUP BY clause. </td>
  </tr>
    <tr>
    <td>5. HAVING Clause</td>
    <td>The HAVING clause applies the condition(s) to the grouped rows specified in the GROUP BY clause, and any grouped rows that aren't satisfied by the condition(s) are discarded.</td>
  </tr>
    <tr>
    <td>6. SELECT Statement</td>
    <td>Expressions specified in the SELECT statement are processed.</td>
  </tr>
    <tr>
    <td>7. ORDER BY Clause </td>
    <td>The ORDER BY clause sorts the rows of data in either ascending (default) or descending order.</td>
  </tr>
</table>

### 2.2 Overview of Data Types
The purpose of a database is to store data. A database contains one or more tables (and other components). Tables consist of columns and rows of data. In the SAS implementation of SQL, the availabe data types are limited to only two possibilities: 
* numeric
* character

#### 2.2.1 Numeric Data
Columns defined as a numeric data type with the NUMERIC or NUM column definition are assigned a default length of 8 bytes, but a `LENGTH=` modifier can be used to assign smaller lengths to a numeric column. 

In [5]:
%%SAS
*Create a table that has numeric columns with a default length of 8 bytes;
proc sql;
    create table purchases
        (custnum  num, 
         prodnum  num, 
         units    num,
         unitcost num(8,2));
quit;

ods noproctitle;
ods select attributes variables;
proc contents data=purchases;
run;

In [6]:
%%SAS
*Create a table that has numeric columns with a specified (smaller) length;
proc sql;
    create table purchases
        (custnum num length=4 
            label='Customer Number', 
         prodnum num length=3
            label='Product Purchased',
         units num length=3
             label='# Units Purchased', 
         unitcost num length=4
             label='Unit Cost');
quit;

ods select attributes variables;
proc contents data=purchases;
run;

#### Date and Time Column Definitions
Date and time values are represented internally as an offset where a SAS date value is stored as the number of days from the fixed date value of 01/01/1960 (January 1, 1960). The SAS date value for January 1, 1960 is represented as 0 (zero). A date earlier than this date is represented as a negative number and a date later than this date is represented as a positive number. 

#### 2.2.2 Character Data
The CHARACTER or CHAR data type stores fixed-length character consisting of a maximum of 32K characters. If a length is not specified, a CHAR column stores a default of 8 charactes. 

### 2.3 SQL Keywords
SQL provides three keywords: `AS`, `DISTINCT`, and `UNIQUE` to perform specific operations on the results. 

#### 2.3.1 Creating Column Aliases
In situations where data are computed using system functions, statistical functions, or arithmetic operations, a column name or header can be left blank. To prevent this from occurring, users may specify the `AS` keyword to provide a name to the column or heading itself along with formatting directions.

In [7]:
%%SAS
proc sql;
    select prodname,
           prodtype,
           prodcost * 0.80 as Discount_Price format=dollar9.2
        from sql.products
            order by 3;
quit;

Product Name,Product Type,Discount_Price
Analog Cell Phone,Phone,$28.00
Office Phone,Phone,$104.00
Digital Cell Phone,Phone,$140.00
Spreadsheet Software,Software,$239.20
Graphics Software,Software,$239.20
Wordprocessor Software,Software,$239.20
Database Software,Software,$319.20
Dream Machine,Workstation,"$2,560.00"
Business Machine,Workstation,"$2,640.00"
Travel Laptop,Laptop,"$2,720.00"


#### 2.3.2 Finding Duplicate (Unique) Values
In some situations, several rows in a table may contain identical or duplicate column values. To select only one of each identical or duplicate values, SAS supports and processes the **DISTINCT** and **UNIQUE keywords** the same and without any noticeable performance differences. 

In [8]:
%%SAS
proc sql;
    select distinct /*unique*/ manunum
        from sql.inventory;
quit;

Manufacturer Number
111
170
500
600


### 2.4 SQL Operators, Functions, and Keywords
There are several types of operators and functions in PROC SQL:
* comparison operators
* logical operators
* arithmetic operators
* character string operators
* summary functions
* predicates
* keywords

#### 2.4.1 Comparison Operators
**Comparison operators** are used in the SQL procedure to compare one character or numeric value to another. 

<table>
  <tr>
    <th>SAS Operator</th>
    <th>Mnemonic Operators</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>=</td>
    <td>EQ</td>
    <td>Equal to</td>
  </tr>
  <tr>
    <td>^= or ~=</td>
    <td>NE</td>
    <td>Not equal to</td>
  </tr>
    <tr>
    <td><</td>
    <td>LT</td>
    <td>Less than</td>
  </tr><tr>
    <td><=</td>
    <td>LE</td>
    <td>Less than or equal to</td>
  </tr><tr>
    <td>></td>
    <td>GT</td>
    <td>Greater than</td>
  </tr><tr>
    <td>>=</td>
    <td>GE</td>
    <td>Greater than or equal to</td>
  </tr>
</table>

In [9]:
%%SAS
*Select only those products from the PRODUCTS table that cost more than $300.00;
proc sql;
    select prodname, 
           prodtype,
           prodcost
        from sql.products
            where prodcost > 300;
quit;

Product Name,Product Type,Product Cost
Dream Machine,Workstation,"$3,200.00"
Business Machine,Workstation,"$3,300.00"
Travel Laptop,Laptop,"$3,400.00"
Database Software,Software,$399.00


#### 2.4.2 Logical Operators
**Logical operators** are used to connect two or more expressions together in a WHERE or HAVING clause. The available logical operators consist of **AND**, **OR**, and **NOT**. 

In [10]:
%%SAS
*Select only those software products that cost more than $300.00;
proc sql;
    select prodname, 
           prodtype,
           prodcost
        from sql.products
            where prodtype = 'Software' and 
                  prodcost > 300;
quit;

Product Name,Product Type,Product Cost
Database Software,Software,$399.00


In [11]:
%%SAS
*Select software products or products that cost more than $300.00;
proc sql;
    select prodname, 
           prodtype, 
           prodcost
        from sql.products;
            where prodtype = 'Software' or 
                  prodcost > 300;
quit;

Product Name,Product Type,Product Cost
Dream Machine,Workstation,"$3,200.00"
Business Machine,Workstation,"$3,300.00"
Travel Laptop,Laptop,"$3,400.00"
Analog Cell Phone,Phone,$35.00
Digital Cell Phone,Phone,$175.00
Office Phone,Phone,$130.00
Spreadsheet Software,Software,$299.00
Database Software,Software,$399.00
Wordprocessor Software,Software,$299.00
Graphics Software,Software,$299.00


In [12]:
%%SAS
*Select products that are not software products and do not cost more than $300.00;
proc sql;
    select prodname,
           prodtype,
           prodcost
        from sql.products;
            where not prodtype = 'Software' and 
                  not prodcost > 300;
quit;

Product Name,Product Type,Product Cost
Dream Machine,Workstation,"$3,200.00"
Business Machine,Workstation,"$3,300.00"
Travel Laptop,Laptop,"$3,400.00"
Analog Cell Phone,Phone,$35.00
Digital Cell Phone,Phone,$175.00
Office Phone,Phone,$130.00
Spreadsheet Software,Software,$299.00
Database Software,Software,$399.00
Wordprocessor Software,Software,$299.00
Graphics Software,Software,$299.00


#### 2.4.3 Arithmetic Operators
<table>
  <tr>
    <th>Operator </th>
    <th>Description</th>
  </tr>
  <tr>
    <td>+</td>
    <td>Addition</td>
  </tr>
  <tr>
    <td>-</td>
    <td>Subtraction</td>
  </tr>
    <tr>
    <td>*</td>
    <td>Multiplication</td>
  </tr>
    <tr>
    <td>/</td>
    <td>Division</td>
  </tr>
    <tr>
    <td>**</td>
    <td>Exponentiation</td>
  </tr>
    <tr>
    <td>=</td>
    <td>Equality</td>
  </tr>
</table>

In [13]:
%%SAS
*Apply a discount of 20% to the product price (PRODCOST) in the PRODUCTS table;
proc sql;
    select prodname, 
           prodtype, 
           prodcost * 0.80 as Dicount_Price
        from sql.products;
quit;

Product Name,Product Type,Dicount_Price
Dream Machine,Workstation,2560
Business Machine,Workstation,2640
Travel Laptop,Laptop,2720
Analog Cell Phone,Phone,28
Digital Cell Phone,Phone,140
Office Phone,Phone,104
Spreadsheet Software,Software,239.2
Database Software,Software,319.2
Wordprocessor Software,Software,239.2
Graphics Software,Software,239.2


PROC SQL allows references to a computed column in the same SELECT statement (or a WHERE clause) using the `CALCULATED` keyword. 

In [14]:
%%SAS
*Apply a discount of 20% to the product price (PRODCOST) in the PRODUCTS table, 
 calculate the loss using the discounted price, and
 sort the resulting set by ascending order of discounted price;
proc sql;
    select prodname,
           prodtype,
           prodcost * 0.80 as Discount_Price
               format=dollar9.2,
           prodcost - calculated Discount_Price as Loss 
               format=dollar7.2
        from sql.products
            order by 3;
run;

Product Name,Product Type,Discount_Price,Loss
Analog Cell Phone,Phone,$28.00,$7.00
Office Phone,Phone,$104.00,$26.00
Digital Cell Phone,Phone,$140.00,$35.00
Spreadsheet Software,Software,$239.20,$59.80
Graphics Software,Software,$239.20,$59.80
Wordprocessor Software,Software,$239.20,$59.80
Database Software,Software,$319.20,$79.80
Dream Machine,Workstation,"$2,560.00",$640.00
Business Machine,Workstation,"$2,640.00",$660.00
Travel Laptop,Laptop,"$2,720.00",$680.00


#### 2.4.4 Character String Operators and Functions

#### Concatenating Strings

In [15]:
%%SAS
*Concatenate two columns and a text string.
 Note that concatenation operator || (or the function cat) 
 put a space before the text string if it is between two columns.
 The two SQL procedures produce the same resulting set;
proc sql;
    select manucity || "," || manustat
        from sql.manufacturers;
quit;

proc sql;
    select cat(manucity, ",", manustat )
        from sql.manufacturers;
quit;

""
"Houston ,TX"
"San Diego ,CA"
"Miami ,FL"
"San Mateo ,CA"
"San Diego ,CA"
"San Diego ,CA"
""
"Houston ,TX"
"San Diego ,CA"
"Miami ,FL"


#### Finding the Length of a String

In [16]:
%%SAS
proc sql;
    select prodnum,
           prodname, 
           length(prodname) as Length
        from sql.products;
quit;

Product Number,Product Name,Length
1110,Dream Machine,13
1200,Business Machine,16
1700,Travel Laptop,13
2101,Analog Cell Phone,17
2102,Digital Cell Phone,18
2200,Office Phone,12
5001,Spreadsheet Software,20
5002,Database Software,17
5003,Wordprocessor Software,22
5004,Graphics Software,17


#### Changing the Case in a String 
#### Finding the Occurrence of a Pattern with INDEX


In [17]:
%%SAS
proc sql;
    select prodnum,
           prodname, 
           prodtype
        from sql.products
            where index(upcase(PRODNAME), 'PHONE') > 0;
quit;

Product Number,Product Name,Product Type
2101,Analog Cell Phone,Phone
2102,Digital Cell Phone,Phone
2200,Office Phone,Phone


#### Extracting Information from a String

In [18]:
%%SAS
proc sql;
    select prodnum, 
           prodname, 
           prodtype, 
           substr(prodtype,1,4)
        from sql.products
            where prodcost > 100;
quit;

Product Number,Product Name,Product Type,
1110,Dream Machine,Workstation,Work
1200,Business Machine,Workstation,Work
1700,Travel Laptop,Laptop,Lapt
2102,Digital Cell Phone,Phone,Phon
2200,Office Phone,Phone,Phon
5001,Spreadsheet Software,Software,Soft
5002,Database Software,Software,Soft
5003,Wordprocessor Software,Software,Soft
5004,Graphics Software,Software,Soft


#### 2.4.5 Summarizing Data
<table>
  <tr>
    <th>Summary Function</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>AVG, MEAN</td>
    <td>Average or mean of values</td>
  </tr>
  <tr>
    <td>COUNT, FREQ, N</td>
    <td>Aggreagate number of non-missing values</td>
  </tr>
  <tr> 
    <td>CV</td>
    <td>Coefficient of variation</td>
  </tr>
  <tr>
    <td>MAX</td>
    <td>Largest value</td>
  </tr>
  <tr>
    <td>MIN</td>
    <td>Smallest value</td>
  </tr>
  <tr>
    <td>NMISS</td>
    <td>Number of missing values</td>
  </tr>
  <tr>
    <td>RANGE</td>
    <td>Difference between the largest and smallest values</td>
  </tr>
  <tr>
    <td>STD</td>
    <td>Standard deviation</td>
  </tr>
  <tr>
    <td>SUM</td>
    <td>Sum of values</td>
  </tr>
  <tr>
    <td>VAR</td>
    <td>Variance</td>
  </tr>
</table>

In [19]:
%%SAS
*Count the total number of missing and nonmissing rows in the PURCHASES table;
proc sql;
    select count(*) as Row_Count
        from sql.purchases;
run;

*Count the total number of nonmissing rows;
proc sql;
    select count(units) as Non_Missing_Row_Count
        from sql.purchases;
run;

Row_Count
57
Non_Missing_Row_Count
57


In [20]:
%%SAS
*What is the least expensive product in the PRODUCTS table?;
proc sql;
    select min(prodcost) as Cheapest
            format=dollar9.2 Label='Least Expensive'
        from sql.products;
quit;

Least Expensive
$35.00


In [21]:
%%SAS
*Determine the total costs of all purchases by customers who bought 
workstation (PRODNUM=1110 and 1200) and laptops (PRODNUM=1700);
proc sql;
    select sum((units) * (unitcost))
            as Total_Purchases format=dollar12.2
        from sql.purchases;
            where prodnum = 1100 or 
                  prodnum = 1200 or 
                  prodnum = 1700;
quit;

Total_Purchases
"$295,473.00"


In [22]:
%%SAS
*Determine the average product for all software in the PRODUCTS table 
 containing a variety of products;
proc sql;
    select avg(prodcost) as 
            Average_Product_Cost format=dollar10.2
        from sql.products;
            where upcase(prodtype) in 
                    ('Software');
quit;

Average_Product_Cost
"$1,153.60"


In [23]:
%%SAS
*Determine the average cost of products in inventory;
proc sql;
    select prodnum,
           (invprice / invqty) as 
                Average_Price
                format=dollar8.2
        from sql.invoice;
quit;

Product Number,Average_Price
5001,$299.00
6001,$799.00
2101,$35.00
1110,$3200.00
5002,$399.00
6000,$99.00
1200,$3300.00


#### 2.4.6 Predicates
**Predicates** are used in PROC SQL to perform direct comparisons between two conditions or expressions.
* **BETWEEN**
* **IN**
* **IS NULL**
* **IS MISSING**
* **LIKE**
* **EXISTS**

In [24]:
%%SAS
*Select the products costing between $200.00 and $500.00 inclusively 
 from the PRODUCTS table;
proc sql;
    select prodname,
           prodtype,
           prodcost
        from sql.products
            where prodcost between 200 and 500;
quit;

Product Name,Product Type,Product Cost
Spreadsheet Software,Software,$299.00
Database Software,Software,$399.00
Wordprocessor Software,Software,$299.00
Graphics Software,Software,$299.00


In [25]:
%%SAS
*Select the products that were ordered between the years 1999 and 2000;
proc sql;
    select prodnum,
           invenqty,
           orddate
        from sql.inventory;
            where year(orddate) between 1999 and 2000;
quit;

Product Number,Inventory Quantity,Date Inventory Last Ordered
1110,20,09/01/2000
1700,10,08/15/2000
5001,5,08/15/2000
5002,3,08/15/2000
5003,10,08/15/2000
5004,20,09/01/2000
5001,2,09/01/2000


In [26]:
%%SAS
*Select the products that wered ordered between 1999 and 200 
 or where inventory quantities are greater than 15;
proc sql;
    select prodnum,
           invenqty,
           orddate
        from sql.inventory
            where (year(orddate) between 1999 and 2000) or
                   invenqty > 15;
quit;

Product Number,Inventory Quantity,Date Inventory Last Ordered
1110,20,09/01/2000
1700,10,08/15/2000
5001,5,08/15/2000
5002,3,08/15/2000
5003,10,08/15/2000
5004,20,09/01/2000
5001,2,09/01/2000


In [27]:
%%SAS
*Select all phone products from the PRODUCTS table;
proc sql;
    select prodname,
           prodtype,
           prodcost
        from sql.products
            where upcase(prodtype) = 'PHONE';
quit;

Product Name,Product Type,Product Cost
Analog Cell Phone,Phone,$35.00
Digital Cell Phone,Phone,$175.00
Office Phone,Phone,$130.00


In [28]:
%%SAS
*Select both phone and software products from the PRODUCTS table;
proc sql;
    select prodname,
           prodtype,
           prodcost
        from sql.products
            where upcase(prodtype) in ('PHONE', 'SOFTWARE');
quit;

Product Name,Product Type,Product Cost
Analog Cell Phone,Phone,$35.00
Digital Cell Phone,Phone,$175.00
Office Phone,Phone,$130.00
Spreadsheet Software,Software,$299.00
Database Software,Software,$399.00
Wordprocessor Software,Software,$299.00
Graphics Software,Software,$299.00


In [29]:
%%SAS
*Select the products that are out-of-stock from the INVENTORY table;
proc sql;
    select prodnum,
           invenqty,
           invencst
        from sql.inventory
            where invenqty is null /*is missing*/;
quit;

In [30]:
%%SAS
*Select the products that are currently stocked from the INVENTORY table;
proc sql;
    select prodnum,
           invenqty,
           invencst
        from sql.inventory
            where invenqty is not null /*is not missing*/;
quit;

Product Number,Inventory Quantity,Inventory Cost
1110,20,"$45,000.00"
1700,10,"$28,000.00"
5001,5,"$1,000.00"
5002,3,$900.00
5003,10,"$2,000.00"
5004,20,"$1,400.00"
5001,2,"$1,200.00"


In [31]:
%%SAS
*Select all product names that start with the upperchase character 'A';
proc sql;
    select prodname
        from sql.products;
            where prodname like 'A%';
quit;

Product Name
Dream Machine
Business Machine
Travel Laptop
Analog Cell Phone
Digital Cell Phone
Office Phone
Spreadsheet Software
Database Software
Wordprocessor Software
Graphics Software


In [32]:
%%SAS
*Select all products whose name contains the word 'Soft' in its name;
proc sql;
    select prodname,
           prodtype,
           prodcost
        from sql.products
            where prodtype like '%Soft%';
quit;

Product Name,Product Type,Product Cost
Spreadsheet Software,Software,$299.00
Database Software,Software,$399.00
Wordprocessor Software,Software,$299.00
Graphics Software,Software,$299.00


In [33]:
%%SAS
*Select all products whose name has trailing blanks;
proc sql;
    select prodname
        from sql.products;
            where prodname like '% ';
quit;

Product Name
Dream Machine
Business Machine
Travel Laptop
Analog Cell Phone
Digital Cell Phone
Office Phone
Spreadsheet Software
Database Software
Wordprocessor Software
Graphics Software


In [34]:
%%SAS
*Select all products whose product type has five letters and 
 starts with an uppercase character 'P';
proc sql;
    select prodname,
           prodtype,
           prodcost
        from sql.products;
            where upcase(prodtype) like 'P____';
quit;

Product Name,Product Type,Product Cost
Dream Machine,Workstation,"$3,200.00"
Business Machine,Workstation,"$3,300.00"
Travel Laptop,Laptop,"$3,400.00"
Analog Cell Phone,Phone,$35.00
Digital Cell Phone,Phone,$175.00
Office Phone,Phone,$130.00
Spreadsheet Software,Software,$299.00
Database Software,Software,$399.00
Wordprocessor Software,Software,$299.00
Graphics Software,Software,$299.00


In [35]:
%%SAS
*Select all products whose name has the lowercase character 'a' in the fourth position;
proc sql;
    select prodname
        from sql.products
            where prodname like '___a%';
quit;

Product Name
Dream Machine
Database Software


In [36]:
%%SAS
*Check for the existence of customers in the CUSTOMERS table with
 purchases from the PURCHASES table;
proc sql;
    select custnum, 
           custname, 
           custcity
        from sql.customers c
            where exists
                (select *
                    from sql.purchases p
                        where c.custnum = p.custnum);
quit;

Customer Number,Customer Name,Customer's Home City
101,La Mesa Computer Land,La Mesa
201,Vista Tech Center,Vista
301,Coronado Internet Zone,Coronado
401,La Jolla Computing,La Jolla
501,Alpine Technical Center,Alpine
701,San Diego Byte Store,San Diego
801,Jamul Hardware & Software,Jamul
901,Del Mar Tech Center,Del Mar
1001,Lakeside Software Center,Lakeside
1101,Bonsall Network Store,Bonsall
